<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## Install

```sh
pip install dddex
conda install dddex
```

## What is dddex?

The package name `dddex` stands for *Data-Driven Density Estimaton x*. New approaches are being implemented for estimating conditional densities without any parametric assumption about the underlying distribution. All those approaches take an arbitrary point forecaster as input and turn them into a new object that outputs an estimation of the conditional density based on the point predictions of the original point forecaster. The *x* in the name emphasizes that the approaches can be applied to any point forecaster. The approaches are being implemented via the two classes [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex) and [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex_knn) whose usage is explained in detail below.

All models can be run easily with only a few lines of code and are designed to be compatible with the well known *Scikit-Learn* framework.

## How to use: LevelSetKDEx

To ensure compatibility with Scikit-Learn, as usual the classes [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex) and [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex_knn) both implement a `fit` and `predict` method. As they compute estimations of conditional densities, the `predict` method outputs p-quantiles rather than point forecasts. 

Our choice of the class-names is supposed to be indicative of the underlying models:  The name *LevelSet* stems from the fact that both methods operate with the underlying assumption that the values of point forecasts generated by the same point forecaster can be interpreted as a similarity measure between samples. *KDE* is short for *Kernel Density Estimator* and the *x* yet again signals that the classes can be initialized with any point forecasting model. 

In the following, we demonstrate how to use the class [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex) to compute estimations of the conditional densities and quantiles for the [Yaz Data Set](https://opimwue.github.io/ddop/modules/auto_generated/ddop.datasets.load_yaz.html#ddop.datasets.load_yaz). As explained above, [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex) is always based on a point forecaster that is being specified by the user. In our example we use the well known `LightGBMRegressor` as the underlying point predictor.

In [ ]:
from lightgbm import LGBMRegressor
from dddex.levelSetKDEx import LevelSetKDEx
from dddex.loadData import loadDataYaz

In [ ]:
dataYaz, XTrain, yTrain, XTest, yTest = loadDataYaz(testDays = 28, returnXY = True)
LGBM = LGBMRegressor(n_jobs = 1)

There are three parameters for [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex):
1. **estimator**: A point forecasting model that must have a *.predict* method.
2. **binSize**: The amount of training samples considered to compute the conditional densities (for more details, see *To be written*).
3. **weightsByDistance**: If *False*, all considered training samples are weighted equally. If *True*, training samples are weighted by the inverse of the distance of their respective point forecast to the point forecast of the test sample at hand.

In [ ]:
LSKDEx = LevelSetKDEx(estimator = LGBM, 
                      binSize = 100,
                      weightsByDistance = False)

There is no need to run `fit` on the point forecasting model before initializing *LevelSetKDEx*, because the `fit` method of [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex) automatically checks whether the provided model has been fitted already or not and runs the respective `fit` method of the point forecaster if needed.

It should be noted, that running `fit` for the *LevelSetKDEx* approaches takes exceptionally little time even for datasets with $>10^6$ samples (provided, of course, that the underlying point forecasting model has been fitted before hand).

In [ ]:
LSKDEx.fit(X = XTrain, y = yTrain)

In order to compute conditional densities for test samples now, we simply run the `getWeights` method.

In [ ]:
conditionalDensities = LSKDEx.getWeights(X = XTest,
                                         outputType = 'summarized')
conditionalDensities[0]

(array([0.49, 0.01, 0.21, 0.01, 0.16, 0.07, 0.04, 0.01]),
 array([0.        , 0.01075269, 0.04      , 0.04878049, 0.08      ,
        0.12      , 0.16      , 0.2       ]))

Here, `conditionalDensities` is a list whose elements correspond to the samples specified via `X`. Every element contains a tuple, whose first entry constitutes probabilities and the second entry corresponding demand values (side note: The demand values have been scaled to lie in $[0, 1]$). In the above example, we can for example see that our model estimates that for the first test sample the demand will be 0 with a probability of 49%.

Like the input argument *outputType* of `getWeights` suggests, we can output the conditional density estimations in various different forms. All in all, there are currently 5 output types specifying how the output for each sample looks like:
1. *all*: An array with the same length as the number of training samples. Each entry represents the probability of each training sample.
2. *onlyPositiveWeights*: A tuple. The first element of the tuple represents the probabilities and the second one the indices of the corresponding training sample. Only probalities greater than zero are returned. Note: This is the most memory and computationally efficient output type.
3. *summarized*: A tuple. The first element of the tuple represents the probabilities and the second one the corresponding value of `yTrain`. The probabilities corresponding to identical values of `yTrain` are aggregated.
4. *cumulativeDistribution*: A tuple. The first element of the tuple represents the probabilities and the second one the corresponding value of `yTrain`.
5. *cumulativeDistributionSummarized*: A tuple. The first element of the tuple represents the probabilities and the second one the corresponding value of `yTrain`. The probabilities corresponding to identical values of `yTrain` are aggregated.

For example, by setting `outputType = 'cumulativeDistributionSummarized'` we can compute an estimation of the conditional cumulative distribution function for each sample. Below, we can see that our model predicts the demand of the first sample to be lower or equal than 0.16 with a probability of 99%.

In [ ]:
cumulativeDistributions = LSKDEx.getWeights(X = XTest,
                                            outputType = 'cumulativeDistributionSummarized')
cumulativeDistributions[0]

(array([0.49, 0.5 , 0.71, 0.72, 0.88, 0.95, 0.99, 1.  ]),
 array([0.        , 0.01075269, 0.04      , 0.04878049, 0.08      ,
        0.12      , 0.16      , 0.2       ]))

We can also compute estimations of quantiles using the `predict` method. The parameter *probs* specifies the quantiles we want to predict.

In [ ]:
predRes = LSKDEx.predict(X = XTest,
                         outputAsDf = True, 
                         probs = [0.1, 0.5, 0.75, 0.99])
print(predRes.iloc[0:6, :].to_markdown())

|    |       0.1 |       0.5 |   0.75 |   0.99 |
|---:|----------:|----------:|-------:|-------:|
|  0 | 0         | 0.0107527 |   0.08 |   0.16 |
|  1 | 0         | 0.08      |   0.12 |   0.2  |
|  2 | 0.04      | 0.0967742 |   0.12 |   0.24 |
|  3 | 0.056338  | 0.12      |   0.16 |   0.28 |
|  4 | 0.04      | 0.0967742 |   0.12 |   0.24 |
|  5 | 0.0666667 | 0.16      |   0.2  |   0.32 |


## How to tune binSize parameter of LevelSetKDEx

`dddex` also comes with the class [`binSizeCV`](https://kaiguender.github.io/dddex/levelsetkdex.html#binsizecv) that tunes the important *binSize* parameter via cross-validation in an efficient manner. The class is designed in a very similar fashion to the cross-validation classes of Scikit-Learn. As such, at first [`binSizeCV`](https://kaiguender.github.io/dddex/levelsetkdex.html#binsizecv)is initialized with all the settings for the cross-validation:
1. **estimatorLSx**: Either an object of class [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex) or [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex_knn)
2. **cvFolds**: An iterable yielding (train, test) splits as arrays of indices
3. **binSizeGrid**: The candidate values of *binSize* to evaluate
4. **probs**: The probabilities for which quantiles are computed and evaluated.
4. **refitPerProb**: If True, for ever probability a fitted copy of *estimatorLSx* with the best binSize for the respective p-quantile is stored in the attribute *bestEstimatorLSx*. If False, only a single fitted copy of *estimatorLSx* is stored with the binSize that yielded the lowest average aggregated costs over all quantile estimations.
5. **n_jobs**: How many cross-validation split results to compute in parallel.

After specifying the settings, `fit` has to be called to compute the results of the cross validation. The performance of every *binSize* candidate value is being evaluated by computing the relative reduction of the pinball loss in comparison to the quantile estimations generated by *SAA* (Sample Average Approximation) for every quantile. 

In [ ]:
from dddex.levelSetKDEx import binSizeCV
from dddex.utils import groupedTimeSeriesSplit

dataTrain = dataYaz[dataYaz['label'] == 'train']
cvFolds = groupedTimeSeriesSplit(data = dataTrain, 
                                 kFolds = 3,
                                 testLength = 28,
                                 groupFeature = 'id',
                                 timeFeature = 'dayIndex')

LSKDEx = LevelSetKDEx(estimator = LGBM,
                      binSize = None,
                      weightsByDistance = False)

CV = binSizeCV(estimatorLSx = LSKDEx,
               cvFolds = cvFolds,
               binSizeGrid = [10, 30, 100, 400, 1000],
               probs = [0.01, 0.25, 0.5, 0.75, 0.99],
               refitPerProb = True,
               n_jobs = 3)

CV.fit(X = XTrain, y = yTrain)

The best value for *binSize* can either be computed for every quantile separately or for all quantiles at once by computing the average cost reduction over all quantiles.

In [ ]:
print(f"Best binSize over all quantiles: {CV.bestBinSize}")
CV.bestBinSize_perProb

Best binSize over all quantiles: 1000


0.01    1000
0.25     100
0.50     200
0.75     200
0.99    1000
dtype: int64

The exact results are also stored as attributes. The easiest way to view the results is given via `cv_results`, which depicts the average results over all cross-validation folds:

In [ ]:
print(CV.cv_results.to_markdown())

|      |    0.01 |     0.25 |      0.5 |     0.75 |    0.99 |
|-----:|--------:|---------:|---------:|---------:|--------:|
|   10 | 4.82018 | 0.882168 | 0.833527 | 0.862054 | 3.33116 |
|   30 | 2.70603 | 0.867827 | 0.814407 | 0.842285 | 2.06373 |
|   50 | 2.25913 | 0.858561 | 0.811041 | 0.836977 | 1.67127 |
|  100 | 1.65191 | 0.857026 | 0.803632 | 0.835323 | 1.81003 |
|  200 | 1.7692  | 0.865253 | 0.799966 | 0.829241 | 1.61465 |
|  400 | 1.64183 | 0.860281 | 0.812806 | 0.837641 | 1.57534 |
| 1000 | 1.54641 | 0.869606 | 0.854369 | 0.88065  | 1.52644 |


The attentive reader will certainly notice that values greater than 1 imply that the respective model performed worse than SAA. This is, of course, simply due to the fact, that we didn't tune the hyperparameters of the underlying `LGBMRegressor` point predictor and instead used the default parameter values. The [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex)classes are able to produce highly accurate density estimations, but are obviously not able to turn a terrible point predictor into a highly performant conditional density estimator. The performance of the underlying point predictor and the constructed [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex) model go hand in hand.

We can also access the results for every fold separately via `cv_results_raw`, which is a list with one entry per fold:

In [ ]:
CV.cv_results_raw

[          0.01      0.25      0.50      0.75      0.99
 10    4.539182  0.894951  0.898096  0.964582  4.735189
 30    3.026137  0.890452  0.864406  0.935107  2.844497
 50    2.171446  0.867088  0.851076  0.921314  2.469819
 100   1.626054  0.871327  0.833379  0.907911  2.591117
 200   1.739099  0.855421  0.826119  0.885716  2.141826
 400   1.732673  0.860440  0.828015  0.890643  2.190292
 1000  1.464534  0.873277  0.858563  0.891858  1.830334,
           0.01      0.25      0.50      0.75      0.99
 10    5.710189  0.855178  0.783143  0.826845  2.653691
 30    3.091404  0.862482  0.813418  0.835087  1.754652
 50    2.760545  0.868976  0.820686  0.823381  1.310648
 100   1.752709  0.862970  0.812126  0.819613  1.416013
 200   2.137508  0.884030  0.817855  0.829791  1.412322
 400   2.085622  0.887758  0.839370  0.859290  1.296445
 1000  1.767468  0.869484  0.860893  0.876293  1.464460,
           0.01      0.25      0.50      0.75      0.99
 10    4.211159  0.896375  0.819344  0.794736 

The models with the best *binSize* parameter are automatically computed while running `fit` and can be accessed via `bestEstimatorLSx`. If `refitPerProb = True`, then `bestEstimatorLSx` is a dictionary whose keys are the probabilities specified via the paramater *probs*.

In [ ]:
LSKDEx_best99 = CV.bestEstimatorLSx[0.99]
predRes = LSKDEx_best99.predict(X = XTest,
                                probs = 0.99)
print(predRes.iloc[0:6, ].to_markdown())

|    |   0.99 |
|---:|-------:|
|  0 |   0.32 |
|  1 |   0.32 |
|  2 |   0.32 |
|  3 |   0.32 |
|  4 |   0.32 |
|  5 |   0.32 |


## Benchmarks: Random Forest wSAA

The `dddex` package also contains useful non-parametric benchmark models to compare the performance of the [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex) models to other state of the art non-parametric models capable of generating conditional density estimations. In a [meta analysis conducted by S. Butler et al.](https://ml-eval.github.io/assets/pdf/ICLR22_Workshop_ML_Eval_DDNV.pdf) the most performant model has been found to be [weighted sample average approximation (wSAA)](https://pubsonline.informs.org/doi/10.1287/mnsc.2018.3253) based on *Random Forest*. This model has been implemented in a Scikit-Learn fashion as well.

In [ ]:
from dddex.wSAA import RandomForestWSAA
RF = RandomForestWSAA()

[`RandomForestWSAA`](https://kaiguender.github.io/dddex/wsaa.html#randomforestwsaa) is a class derived from the original `RandomForestRegressor` class from Scikit-Learn, that has been extended to be able to generate conditional density estimations in the manner described by Bertsimas et al. in their paper [*From Predictive to prescriptive analytics*](https://pubsonline.informs.org/doi/10.1287/mnsc.2018.3253). The *Random Forest* modell is being fitted in exactly the same way as the original *RandomForestRegressor*:

In [ ]:
RF.fit(X = XTrain, y = yTrain)

Identical to the [`LevelSetKDEx`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex) and [`LevelSetKDEx_kNN`](https://kaiguender.github.io/dddex/levelsetkdex.html#levelsetkdex_knn) classes, an identical method called `getWeights` and `predict`are implemented to compute conditional density estimations and quantiles. The output is the same as before.

In [ ]:
conditionalDensities = RF.getWeights(X = XTest,
                                     outputType = 'summarized')
conditionalDensities[0]

(array([0.09648016, 0.19948413, 0.26200397, 0.15707937, 0.17230952,
        0.05983333, 0.03134127, 0.01392857, 0.00611111, 0.00142857]),
 array([0.  , 0.04, 0.08, 0.12, 0.16, 0.2 , 0.24, 0.28, 0.32, 0.44]))

In [ ]:
predRes = RF.predict(X = XTest,
                     outputAsDf = True)
print(predRes.iloc[0:6, :].to_markdown())

|    |   0.1 |   0.5 |   0.9 |
|---:|------:|------:|------:|
|  0 |  0.04 |  0.08 |  0.2  |
|  1 |  0.04 |  0.12 |  0.24 |
|  2 |  0.04 |  0.12 |  0.24 |
|  3 |  0.04 |  0.16 |  0.24 |
|  4 |  0.04 |  0.12 |  0.28 |
|  5 |  0.08 |  0.2  |  0.32 |


The original `predict` method of the `RandomForestRegressor` has been renamed to `pointPredict`:

In [ ]:
RF.pointPredict(X = XTest)[0:6]

array([0.1024    , 0.1328    , 0.1392    , 0.1524    , 0.1472    ,
       0.1944    , 0.25751982, 0.1444    , 0.1348    , 0.1548    ])